In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pickle
import gentrl
import numpy as np

import susi
from susi.SOMPlots import plot_estimation_map, plot_som_histogram, plot_umatrix
import gentrl
import torch
torch.cuda.set_device(0)

In [20]:
model.cuda();

In [2]:
enc = gentrl.RNNEncoder(latent_size=50)
dec = gentrl.DilConvDecoder(latent_input_size=50)
model = gentrl.GENTRL(enc, dec, 50 * [('c', 20)], [('c', 20)], beta=0.001)
model.cuda();

C:\Users\mored\Miniconda3\envs\leishnn\lib\site-packages\torch\nn\modules\container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


In [17]:
# Load from past
model.load('ligase_gentrl/')

In [4]:
som_general = pickle.load(open("general_leish_som_500K.p", "rb"))
som_specific = pickle.load(open("specific_leish_som_800K_2.p", "rb"))

In [16]:
from moses.metrics import mol_passes_filters, SA, logP
from moses.metrics.utils import get_n_rings, get_mol

from moses.utils import disable_rdkit_log
disable_rdkit_log()

def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])


def penalized_logP(mol_or_smiles, masked=False, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    smiles_tokenized = gentrl.tokenizer.encode([mol_or_smiles])[0][0].tolist()
    general_award = 0.015*som_general.predict([smiles_tokenized])[0]
    specific_award = 1.4*som_specific.predict([smiles_tokenized])[0]
    reward = logP(mol) - 1.2*SA(mol) - get_num_rings_6(mol) + specific_award + general_award
    if masked and not mol_passes_filters(mol):
        return default
    return reward

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [49]:
model.train_as_rl(penalized_logP)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -4.143;valid_perc: 0.1784;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -4.105;valid_perc: 0.1832;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -4.092;valid_perc: 0.1845;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -4.061;valid_perc: 0.1901;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -4.033;valid_perc: 0.1953;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.998;valid_perc: 0.2024;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.999;valid_perc: 0.2024;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.971;valid_perc: 0.2074;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.904;valid_perc: 0.2178;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.902;valid_perc: 0.2191;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.888;valid_perc: 0.2178;
!!!!!!!!!!!

KeyboardInterrupt: 

In [52]:
model.train_as_rl(penalized_logP)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.84;valid_perc: 0.9141;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.99;valid_perc: 0.9206;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.92;valid_perc: 0.9179;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.8;valid_perc: 0.9135;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.91;valid_perc: 0.917;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.81;valid_perc: 0.9128;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.65;valid_perc: 0.9067;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.77;valid_perc: 0.9108;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.69;valid_perc: 0.9087;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.87;valid_perc: 0.9155;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: 16.79;valid_perc: 0.912;
!!!!!!!!!!!!!!!!!!!!!!!!!

KeyboardInterrupt: 

In [21]:
model.train_as_rl(penalized_logP,verbose_step=25)

!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -4.034;valid_perc: 0.2315;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -4.013;valid_perc: 0.2274;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.934;valid_perc: 0.2434;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.973;valid_perc: 0.2344;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -4.0;valid_perc: 0.2286;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.959;valid_perc: 0.2376;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.94;valid_perc: 0.2408;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.919;valid_perc: 0.2426;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.95;valid_perc: 0.2364;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.911;valid_perc: 0.2466;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.924;valid_perc: 0.2424;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.881;valid_perc: 0.2472;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.881;valid_perc: 0.248;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.88;valid_perc: 0.2492;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -3.855;valid_perc: 0.2572;
!!!!!!!!!!!!!!!!!!!!!!!!!mean_re

KeyboardInterrupt: 